# Project 1 Development Notebook: Climate and Housing

This file contains development code for project 1 for ECON 1680. It contains all code required to replicate the results outlined in the draft. Inline citations are included for all code segments that are not original. The code first cleans the data, generates some descriptive statistics, and then leverages the methodology discussed in the paper: 
We use different regressions and dimension reduction techniques to analyze the relationship between climate risk and housing prices, using space as our source of variation.

## 1. Imports and Functions

In [303]:
import pandas as pd
import numpy as np
from dateutil.parser import parse

In [304]:
def is_date(string):
    # From https://stackoverflow.com/questions/25341945/check-if-string-has-date-any-format
    #   Checks if a string can be interpreted as a date
    try:
        parse(string)
        return True
    except ValueError:
        return False

In [305]:
def get_pct_change(row, start_date, end_date):
    # This function gets the percentage change in ZHVI from 2000-2019 for a given county.
    #   We use this later to generate one of our dependent variables.
    pct_change = (row[end_date] - row[start_date]) / row[start_date]
    return pct_change

In [306]:
def get_volatility(row, date_vars):
    # This function average of the yearly coefficients of variation in ZHVI in a county.
    coef_of_vars = []
    for n in range(0, len(date_vars), 12):
        year_vars = date_vars[n:n + 12]
        year_price_data = row[year_vars].tolist()

        # Drop year observations if we don't have sufficient data to calculate the coefficient of variation.
        if True not in [np.isnan(i) for i in year_price_data]:
            coef_of_vars.append(np.std(year_price_data) / np.mean(year_price_data))
        
    # If we have no information about volatility, return missing.
    if coef_of_vars == []:
        return np.nan
    return np.mean(coef_of_vars)

## 2. Data: Cleaning and Saving

In [307]:
# Path to production directory: note that you must include Data, Figures, and Code subdirectories.
path = "C:\\Users\\garvg\\Downloads\\Project 1\\ClimateAndHousing\\"

# Read in CSV data
zillow = pd.read_csv(path + "Data\\zillow.csv")
nri_fema = pd.read_csv(path + "Data\\nri_fema.csv")
oi_covars = pd.read_csv(path + "Data\\oi_covars.csv")

In [308]:
# Merge CSV data by State and County FIPS codes and save
clim_hous_data = zillow.merge(nri_fema, on=["state_fips", "county_fips"], validate="one_to_one")
all_data = clim_hous_data.merge(oi_covars, on=["state_fips", "county_fips"], validate="one_to_one")
all_data.to_csv(path + "Data\\all_data.csv", index=False)

In [309]:
# Specify columns to keep: hazard scores, controls, and house value
hazard_vars = [
    "Avalanche", "Coastal Flooding", "Cold Wave", "Drought", "Earthquake", 
    "Hail", "Heat Wave", "Hurricane", "Ice Storm", "Landslide", "Lightning",
    "Riverine Flooding", "Strong Wind", "Tornado", "Tsunami", "Volcanic Activity",
    "Wildfire", "Winter Weather"]
hazard_vars = [hazard + " - Hazard Type Risk Index Score" for hazard in hazard_vars]
hazard_vars.append("National Risk Index - Score - Composite")

housing_vars = [col for col in all_data.columns if is_date(col) and "202" not in col]
housing_vars.sort()

control_vars = [
    "Population (2020)", "Building Value ($)", "Agriculture Value ($)", "Area (sq mi)", "job_density_2013",
    "ann_avg_job_growth_2004_2013", "ln_wage_growth_hs_grad", "emp2000", "foreign_share2010", 
    "mean_commutetime2000", "frac_coll_plus2000", "frac_coll_plus2010", "hhinc_mean2000",
    "med_hhinc1990", "med_hhinc2016", "poor_share1990", "poor_share2000", "poor_share2010",
    "share_white2000", "share_black2000", "share_hisp2000", "share_asian2000",
    "share_white2010", "share_black2010", "share_hisp2010", "share_asian2010"
    ]

columns = ["State Name", "County Name", "state_fips", "county_fips"]
columns.extend(hazard_vars)
columns.extend(housing_vars)
columns.extend(control_vars)

all_data_pruned = all_data[columns]

In [310]:
# For hazard data, rename variables and replace missing values with zeroes:
#   Missing values imply that "Community is not considered at risk for hazard type."
#   according to the official NRI documentation here:
#   https://www.fema.gov/sites/default/files/documents/fema_national-risk-index_technical-documentation.pdf
hazard_rename_dict = dict()

for hazard in hazard_vars:
    hazard_rename_dict[hazard] = hazard[:hazard.index(" - ")]
    all_data_pruned[hazard] = all_data_pruned[hazard].fillna(0)

all_data_pruned = all_data_pruned.rename(hazard_rename_dict, axis=1)

C:\Users\garvg\AppData\Local\Temp\ipykernel_13828\2145004020.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_pruned[hazard] = all_data_pruned[hazard].fillna(0)


### Dependent Variable Generation

The NRI takes into account historical loss data from 1996-2019 when computing natural hazard risk. We can thus take the NRI as a measure of both hazard risk and historical hazard loss. This can be supported by the documentation, which is included here: https://www.fema.gov/sites/default/files/documents/fema_national-risk-index_technical-documentation.pdf. This will be corroborated later, as we will show that the incidence of certain natural disasters in South Carolina is strongly correlated with these risk indices.

We bring this up to explain that we will consider Zillow home value data (the Zillow Home Value Index, or ZHVI is our measure) from the years 2000 (earliest available) to 2019. Our dependent variables will thus be the following:
1. Mean house value over this period (to capture the relationship between risk and magnitude of housing prices)
2. Average percentage change in housing price (to capture the relationship between risk and long-term upward/downward trends)
3. Average within-year variance in housing price (to capture the relationship between risk and short-term housing price volatility)

We generate these values below.

In [311]:
# Generate mean ZHVI over 2000-2019
all_data_pruned["ZHVI_mean"] = all_data_pruned[housing_vars].mean(axis=1)

# Generate mean percentage change over 2000-2019
all_data_pruned["ZHVI_trend"] = all_data_pruned.apply(get_pct_change, args=("2000-01-31", "2019-12-31"), axis=1)

# Generate mean within-year (month-to-month) variance over 2000-2019
all_data_pruned["ZHVI_vol"] = all_data_pruned.apply(get_volatility, args=(housing_vars,), axis=1) 

In [312]:
# Drop unnecessary variables
all_data_pruned = all_data_pruned.drop(columns=housing_vars)

# Save data after cleaning
all_data_pruned.to_csv(path + "Data\\all_data_pruned.csv", index=False)

## 3. Descriptive Statistics

Note: After running the above code, we may always start here since the pre-processing steps are saved in the all_data_pruned.csv file.

In [313]:
# Load data
working_data = pd.read_csv(path + "Data\\all_data_pruned.csv")
working_data.head()

,State Name,County Name,state_fips,county_fips,Avalanche,Coastal Flooding,Cold Wave,Drought,Earthquake,Hail,...,share_black2000,share_hisp2000,share_asian2000,share_white2010,share_black2010,share_hisp2010,share_asian2010,ZHVI_mean,ZHVI_trend,ZHVI_vol
0,California,Los Angeles,6,37,33.653846,43.259557,0.000000,73.846643,100.000000,48.106904,...,0.100307,0.439193,0.105664,0.277873,0.089271,0.477450,0.121261,421124.410815,2.057027,0.034877
1,Illinois,Cook,17,31,0.000000,44.265594,100.000000,19.949093,96.659243,93.923003,...,0.242277,0.196171,0.046331,0.438595,0.249698,0.239623,0.056553,197042.801648,0.587883,0.020780
2,Texas,Harris,48,201,0.000000,73.843058,99.204582,88.450525,90.741330,94.050270,...,0.176150,0.307376,0.045832,0.329789,0.189457,0.408444,0.051969,140117.773455,0.856429,0.013150
3,Arizona,Maricopa,4,13,0.000000,0.000000,0.000000,85.841553,98.218263,99.331849,...,0.037874,0.243300,0.017987,0.586845,0.054242,0.295705,0.030152,206654.847968,1.082801,0.034382
4,California,San Diego,6,73,31.250000,32.796781,0.000000,89.564111,99.745466,22.589882,...,0.060665,0.260778,0.083679,0.484619,0.055997,0.320274,0.098978,424881.784092,1.785324,0.030891


In [314]:
# Generate map of hazard risk

# Generate map of housing prices

# Generate map of housing price trend

# Generate map of housing price volatility


In [338]:
# Write descriptive statistics to logfile
summary_statistics = working_data.describe()
summary_statistics.to_csv(path + '\\Figures\\summary_statistics.csv', index_label='Metric')
summary_statistics.head()

,state_fips,county_fips,Avalanche,Coastal Flooding,Cold Wave,Drought,Earthquake,Hail,Heat Wave,Hurricane,...,share_black2000,share_hisp2000,share_asian2000,share_white2010,share_black2010,share_hisp2010,share_asian2010,ZHVI_mean,ZHVI_trend,ZHVI_vol
count,3073.000000,3073.000000,3073.000000,3073.000000,3073.000000,3073.000000,3073.000000,3073.000000,3073.000000,3073.000000,...,3073.000000,3073.000000,3050.000000,3073.000000,3073.000000,3073.000000,3059.000000,3005.000000,1034.000000,2988.000000
mean,30.279857,103.366417,3.230056,7.867445,46.351206,49.280821,50.595847,50.327553,49.254025,36.173363,...,0.088648,0.060585,0.007259,0.786937,0.094079,0.082362,0.009541,138742.559884,0.895186,0.016478
std,15.063746,107.779895,14.412471,21.809221,33.456454,30.498124,28.676059,28.770805,30.869760,33.256354,...,0.142850,0.116709,0.019129,0.192676,0.145760,0.129940,0.021505,80068.204022,0.418654,0.006075
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000820,0.000000,0.028604,0.000000,0.001311,0.000000,24942.839331,-0.134327,0.002858
25%,19.000000,35.000000,0.000000,0.000000,0.000000,25.167038,25.930640,25.389755,26.057906,0.000000,...,0.004115,0.009270,0.001705,0.673463,0.007189,0.016089,0.002425,91076.527040,0.598596,0.012313


## 4. Regressions

We use standard OLS estimation, LASSO, and Ridge regressions to capture both what variables are the most correlated and predictive of long-term trends, average magnitude, and short-term volatility in local housing prices.

In [316]:
# Create lists of independent (hr), dependent (y), and control (X) variables for analysis
hr_vars = [hazard_rename_dict[i] for i in hazard_vars]
y_vars = ["ZHVI_mean", "ZHVI_trend", "ZHVI_vol"]
X_vars = control_vars

### OLS

We use this approach as a baseline estimator of the relationships between hazard risk and housing price characteristics.

In [317]:
# Conduct OLS Regression on mean ZHVI (without controls)

In [318]:
# Conduct OLS Regression on mean ZHVI (with controls)

In [319]:
# Conduct OLS Regression on change in ZHVI (without controls)

In [320]:
# Conduct OLS Regression on change in ZHVI (with controls)

In [321]:
# Conduct OLS Regression on mean yearly variance in ZHVI (without controls)

In [322]:
# Conduct OLS Regression on mean yearly variance in ZHVI (with controls)

### LASSO

We conduct this analysis to assess variable selection and dimension reduction, particularly in the context of what hazards are important in predicting ZHVI characteristics.

In [323]:
# Conduct LASSO Regression on mean ZHVI (without controls)

In [324]:
# Conduct LASSO Regression on mean ZHVI (with controls)

In [325]:
# Conduct LASSO Regression on change in ZHVI (without controls)

In [326]:
# Conduct LASSO Regression on change in ZHVI (with controls)

In [327]:
# Conduct LASSO Regression on mean yearly variance in ZHVI (without controls)

In [328]:
# Conduct LASSO Regression on mean yearly variance in ZHVI (with controls)

### Ridge

We conduct this analysis for the same reason as LASSO: to assess variable selection and dimension reduction.

In [329]:
# Conduct Ridge Regression on mean ZHVI (without controls)

In [330]:
# Conduct Ridge Regression on mean ZHVI (with controls)

In [331]:
# Conduct Ridge Regression on change in ZHVI (without controls)

In [332]:
# Conduct Ridge Regression on change in ZHVI (with controls)

In [333]:
# Conduct Ridge Regression on mean yearly variance in ZHVI (without controls)

In [334]:
# Conduct Ridge Regression on mean yearly variance in ZHVI (with controls)